In [1]:
dataset = read.csv('../../pynb/08-deep-learning/Churn_Modelling.csv')

In [2]:
dataset = dataset[4:14]

In [4]:
names(dataset)

[1] "CreditScore"     "Geography"       "Gender"          "Age"            
 [5] "Tenure"          "Balance"         "NumOfProducts"   "HasCrCard"      
 [9] "IsActiveMember"  "EstimatedSalary" "Exited"

In [5]:
unique(dataset$Geography)
unique(dataset$Gender)

[1] France  Spain   Germany
Levels: France Germany Spain

[1] Female Male  
Levels: Female Male

In [6]:
nrow(dataset)
nrow(training_set)
nrow(test_set)


[1] 10000

ERROR: Error in nrow(training_set): object 'training_set' not found


In [7]:
ncol(dataset)
ncol(training_set)
ncol(test_set)


[1] 11

ERROR: Error in ncol(training_set): object 'training_set' not found


In [8]:
sapply(training_set, typeof)

ERROR: Error in lapply(X = X, FUN = FUN, ...): object 'training_set' not found


In [9]:
dataset$Geography = as.numeric(factor(dataset$Geography,
                           levels = c('France', 'Spain', 'Germany'),
                           labels = c(1,2,3)))
                                       

In [10]:
dataset$Gender = as.numeric(factor(dataset$Gender,
                           levels = c('Female', 'Male'),
                           labels = c(1,2)))
                                     

In [11]:
unique(dataset$Geography)
unique(dataset$Gender)

[1] 1 2 3

[1] 1 2

In [12]:
sapply(training_set, typeof)

ERROR: Error in lapply(X = X, FUN = FUN, ...): object 'training_set' not found


In [13]:
library('caTools')
set.seed(123)
split = sample.split(dataset$Exited, SplitRatio=0.8 )
training_set = subset(dataset, split == TRUE)
test_set = subset(dataset, split==FALSE)

In [14]:
#Feature Scaling
training_set[-11] = scale(training_set[-11])
test_set[-11] = scale(test_set[-11])

In [15]:
#install.packages('xgboost')

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [26]:
library(xgboost)

In [28]:
classifier = xgboost(data = as.matrix(training_set[-11]), label = training_set$Exited, nrounds =10)

[1]	train-rmse:0.417725 
[2]	train-rmse:0.369588 
[3]	train-rmse:0.342099 
[4]	train-rmse:0.325681 
[5]	train-rmse:0.316159 
[6]	train-rmse:0.310498 
[7]	train-rmse:0.305414 
[8]	train-rmse:0.303013 
[9]	train-rmse:0.300684 
[10]	train-rmse:0.298272 


In [30]:
y_pred = predict(classifier, newdata = as.matrix(test_set[-11]))
y_pred = (y_pred >= 0.5)

In [32]:
cm = table(test_set[, 11], y_pred)
cm

   y_pred
    FALSE TRUE
  0  1540   53
  1   212  195

In [36]:
library(caret)
folds = createFolds(training_set$Exited,k = 10)

cv = lapply(folds, function(x){
    training_fold = training_set[-x,]
    test_fold = training_set[-x,]
    classifier = xgboost(data = as.matrix(training_fold[-11]), label = training_fold$Exited, nrounds =10)
    y_pred = predict(classifier, newdata = as.matrix(test_fold[-11]))
    y_pred = (y_pred >= 0.5)
    cm = table(test_fold[, 11], y_pred)
    acc = ( cm[1,1] + cm[2,2]) / (cm[1,1] + cm[1,2] + cm[2,1] + cm[2,2])
    return (acc)
})

[1]	train-rmse:0.417201 
[2]	train-rmse:0.368832 
[3]	train-rmse:0.340964 
[4]	train-rmse:0.325595 
[5]	train-rmse:0.316141 
[6]	train-rmse:0.308926 
[7]	train-rmse:0.302852 
[8]	train-rmse:0.299579 
[9]	train-rmse:0.297938 
[10]	train-rmse:0.296049 
[1]	train-rmse:0.417598 
[2]	train-rmse:0.369305 
[3]	train-rmse:0.341342 
[4]	train-rmse:0.325526 
[5]	train-rmse:0.316261 
[6]	train-rmse:0.309897 
[7]	train-rmse:0.304197 
[8]	train-rmse:0.300251 
[9]	train-rmse:0.298143 
[10]	train-rmse:0.296899 
[1]	train-rmse:0.417369 
[2]	train-rmse:0.369523 
[3]	train-rmse:0.341448 
[4]	train-rmse:0.325626 
[5]	train-rmse:0.315721 
[6]	train-rmse:0.308621 
[7]	train-rmse:0.304177 
[8]	train-rmse:0.300091 
[9]	train-rmse:0.297148 
[10]	train-rmse:0.294812 
[1]	train-rmse:0.416846 
[2]	train-rmse:0.367994 
[3]	train-rmse:0.339346 
[4]	train-rmse:0.322414 
[5]	train-rmse:0.313346 
[6]	train-rmse:0.306553 
[7]	train-rmse:0.300859 
[8]	train-rmse:0.297184 
[9]	train-rmse:0.294213 
[10]	train-rmse:0.2916

In [37]:
acc = mean(as.numeric(cv))
acc

[1] 0.8850139